In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torchmetrics

In [2]:
# mean and sd for images size 299,299
mean = [0.1854, 0.1854, 0.1855]
std = [0.2005, 0.2005, 0.2005]

batch_size = 32
model_path = '../models/inception_v3_5.pth'

# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
data_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])
test_dataset = datasets.ImageFolder(root="../data/Testing", transform=data_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [20]:
# Load Inception V3 Model
model = torch.jit.load('../models/inception_v3_5.pt')
model.to(device)
model.eval()
print(end="")

# Initi Metrics
accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=len(test_dataset.classes)).to(device)
auc_metric = torchmetrics.AUROC(task="multiclass", num_classes=len(test_dataset.classes)).to(device)
confusion_matrix_metric = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=len(test_dataset.classes)).to(device)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs, _ = model(images)
        
        _, preds = torch.max(outputs, 1)

        accuracy_metric.update(preds, labels)

        auc_metric.update(torch.nn.functional.softmax(outputs, dim=1), labels)

        confusion_matrix_metric.update(preds, labels)
final_accuracy = accuracy_metric.compute()
final_auc = auc_metric.compute()
final_confusion_matrix = confusion_matrix_metric.compute()

print(f"Test Accuracy: {final_accuracy:.4f}")
print(f"Test AUC: {final_auc:.4f}")
print(f"Confusion Matrix:\n{final_confusion_matrix}")

code/__torch__/torchvision/models/inception.py:41: UserWarning: Scripted Inception3 always returns Inception3 Tuple
code/__torch__/torchvision/models/inception.py:41: UserWarning: Scripted Inception3 always returns Inception3 Tuple
code/__torch__/torchvision/models/inception.py:41: UserWarning: Scripted Inception3 always returns Inception3 Tuple
code/__torch__/torchvision/models/inception.py:41: UserWarning: Scripted Inception3 always returns Inception3 Tuple
code/__torch__/torchvision/models/inception.py:41: UserWarning: Scripted Inception3 always returns Inception3 Tuple
code/__torch__/torchvision/models/inception.py:41: UserWarning: Scripted Inception3 always returns Inception3 Tuple
code/__torch__/torchvision/models/inception.py:41: UserWarning: Scripted Inception3 always returns Inception3 Tuple
code/__torch__/torchvision/models/inception.py:41: UserWarning: Scripted Inception3 always returns Inception3 Tuple
code/__torch__/torchvision/models/inception.py:41: UserWarning: Scripted

Test Accuracy: 0.9741
Test AUC: 0.9986
Confusion Matrix:
tensor([[280,  19,   0,   1],
        [  1, 298,   5,   2],
        [  0,   1, 404,   0],
        [  1,   4,   0, 295]], device='cuda:0')


In [28]:
# Load Inception V3 Model
model = models.inception_v3(weights=None)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, len(test_dataset.classes))
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

# Initi Metrics
accuracy_metric = torchmetrics.Accuracy(task="multiclass", num_classes=len(test_dataset.classes)).to(device)
auc_metric = torchmetrics.AUROC(task="multiclass", num_classes=len(test_dataset.classes)).to(device)
confusion_matrix_metric = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=len(test_dataset.classes)).to(device)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        
        _, preds = torch.max(outputs, 1)

        accuracy_metric.update(preds, labels)

        auc_metric.update(torch.nn.functional.softmax(outputs, dim=1), labels)

        confusion_matrix_metric.update(preds, labels)
final_accuracy = accuracy_metric.compute()
final_auc = auc_metric.compute()
final_confusion_matrix = confusion_matrix_metric.compute()

print(f"Test Accuracy: {final_accuracy:.4f}")
print(f"Test AUC: {final_auc:.4f}")
print(f"Confusion Matrix:\n{final_confusion_matrix}")

c:\Users\jesdi\.conda\envs\ml\lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


Test Accuracy: 0.3089
Test AUC: 0.6593
Confusion Matrix:
tensor([[  0,   0, 300,   0],
        [  0,   0, 306,   0],
        [  0,   0, 405,   0],
        [  0,   0, 300,   0]], device='cuda:0')


In [24]:
outputs.logits.shape

torch.Size([31, 4])

In [27]:
outputs.shape

torch.Size([31, 4])